# Обзор игровой индустрии начала XXI века


- Автор: Олеся Макарова

### Цели и задачи проекта
Цель проекта - изучить данные о развитии игровой индустрии с 2000 по 2013 год, структурировать информацию и  выделить наиболее популярные платформы. Для анализа будем использовать датасет `new_games.csv`, в котором содержатся истрические данные, собранные из открытых истотчников.


### Описание данных

Файл `new_games.csv` содержит информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр. Описание данных:

- `Name` — название игры.  
- `Platform` — название платформы.  
- `Year of Release` — год выпуска игры.  
- `Genre` — жанр игры.  
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).  
- `EU sales` — продажи в Европе (в миллионах проданных копий).  
- `JP sales` — продажи в Японии (в миллионах проданных копий).  
- `Other sales` — продажи в других странах (в миллионах проданных копий).  
- `Critic Score` — оценка критиков (от 0 до 100).  
- `User Score` — оценка пользователей (от 0 до 10).  
- `Rating` — рейтинг организации ESRB (*Entertainment Software Rating Board*). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.


### Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод



## 1. Загрузка данных и знакомство с ними


- **Импортируем необходимые библиотеки**:

In [1]:
import pandas as pd
import numpy as np

- **Загрузим необходимые данные из датасета `/datasets/new_games.csv`**:

In [2]:
file_path = "/datasets/new_games.csv"  #локальный файл
file_url = "https://code.s3.yandex.net/datasets/new_games.csv"

try:
    df = pd.read_csv(file_path)  #загружаем локальный файл
    print("Файл загружен локально")
except FileNotFoundError:
    try:
        df = pd.read_csv(file_url) #если локальный не загрузился то загрузим с URL
        print("Файл загружен с URL")
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")

Файл загружен с URL


- **Выведем первые 5 строк датафрейма, чтобы визуально познакомиться с датасетом**:

In [3]:
df.head(5)

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


- **Выводим на экран информацию о датасете (кол-во строк, пропусков, типы данных, вес)**:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


Датасет `new_games.csv` содержит 11 столбцов и 16 956 строк, в которых представлена информация об играх, их рейтинге и продажах. По итогам предварительного анализа, можно отметить следующее:

1. Наименования столбцов не удобны для анализа, рекомендуется привести их к стилю  snake case.
2. Пропуски значений обнаружены в шести столбцах (`Name`,`Year of Release`,`Genre`,`Critic Score` ,  `User Score`,`Rating  `. Необходимо изучить и обработать их перед анализом.
3. Типы данных требуют корректировки:
  - `Year of Release` (float64 → int64) — содержит только год.
  - `EU sales`, `JP sales`, `User Score` (object → float64) — содержат числовые значения.
  - Остальные столбцы имеют верный тип данных.
4. Рекомендуется провести дополнительную проверку на наличие дубликатов.


---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия столбцов датафрейма

- **Приведем наименования столбцов к стилю snake case**:

In [5]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.columns = df.columns.str.strip()

df.head(1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E


### 2.2. Типы данных


**Изучим типы данных и их корректность**: в датасете представлено четыре столбца с типом данных `float64` и семь столбцов с типом данных `object`, однако не для всех столбцов типы данных определены корректно, это может объясняться наличием пропусков или текстовых значений в этих столбцах. 

Рассмотрим более подробно:
- `name` - содержит строковую информацию в виде названия игры, что логично для текстовых данных. Здесь тип данных `object` подходит.
- `platform`-  содержит строковую информацию в виде названия игры. Тип данных `object` установлен верно.
- `year_of_release` - определен как `float64` но хранит в себе информацию о годе выхода игры. Так как данные содержат год (целое число), рекомендуется изменить тип данных на `int64`, предварительно удалив или преобразовав пропуски в этом столбце.
- `genre` - содержит строковую информацию в виде названия жанра. Тип данных `object` установлен верно.
- `na_sales` - определен как `float64` это верное решение так как данные о продажах могут содержать дробные значения.
- `eu_sales` и `jp_sales` - определены как `object` но содержат данные о продажах - числовую информацию в виде дробных чисел. Рекомендуется изменить тип данных на `float64`.
- `other_sales` и `critic_score` - содержат числовую информацию в виде дробных чисел, что соответствует типу данных  `float64` - тип данных определен верно. 
- `user_score` - содержит числовую информацию в виде дробных чисел но определены как `object`, что не соотвтетствует содержанию этого стобца. Рекомендуется изменить тип данных на `float64`.
- `rating` - содержит строковую информацию - букву рейтинга. Тип данных `object` установлен верно.


- **Заменим на `float64` все текстовые значения и символы в колонках с числовыми данными и заменим их на NaN**:

In [6]:
numeric_columns = df.select_dtypes(include=['number']).columns   #получаем список числовых столбцов
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')  #преобразуем числовые столбцы в числовой тип (float64), заменяя ошибки на NaN

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16956 non-null  object 
 6   jp_sales         16956 non-null  object 
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       10152 non-null  object 
 10  rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


- **Заменим тип данных в `year_of_release` на `int64`**:

In [7]:
for column in ['year_of_release']:
    df[column] = df[column].astype('Int64')  
    
df.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  Int64  
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16956 non-null  object 
 6   jp_sales         16956 non-null  object 
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       10152 non-null  object 
 10  rating           10085 non-null  object 
dtypes: Int64(1), float64(3), object(7)
memory usage: 1.4+ MB


- **Заменим тип данных в `eu_sales`,  `jp_sales`,  `user_score` на `float64`**:

In [8]:
columns_to_convert = ['eu_sales', 'jp_sales', 'user_score']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce').astype('float64') #заменим все текстовые значения и символы на NaN.

#проверим все ли колокни приведены к соответствущим типам данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  Int64  
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.4+ MB


### 2.3. Наличие пропусков в данных


- **Найдем пропущенные значения и посчитаем долю пропусков в каждой колонке от общего количества**:

In [9]:
df.isna().sum().sort_values(ascending=False) #считаем сумму пропусков  

user_score         9268
critic_score       8714
rating             6871
year_of_release     275
eu_sales              6
jp_sales              4
name                  2
genre                 2
platform              0
na_sales              0
other_sales           0
dtype: int64

In [10]:
(df.isna().sum().sort_values(ascending=False) / len(df) * 100).round(2) #считаем доли 

user_score         54.66
critic_score       51.39
rating             40.52
year_of_release     1.62
eu_sales            0.04
jp_sales            0.02
name                0.01
genre               0.01
platform            0.00
na_sales            0.00
other_sales         0.00
dtype: float64

**В данных наблюдаются пропуски в восьми столбцах:**

- `user_score`: в 9268 строках (54.66% данных) отсутствует информация об оценке пользователей. Аналогично critic_score, не все игры могли получить пользовательские оценки. Можно заменить на медиану по жанру.
- `critic_score`: в 8714 строках (51.39% данных) отсутствуют данные об оценке критиков. Вероятно, не все игры, особенно старые, оценивались критиками. Можно заменить на медиану по жанру.
- `rating`: в 6871 строках (40.52% данных) отсутствует информация о рейтинге ESRB. Возможно, некоторые игры не получили ESRB-рейтинг. Можно заменить на «no rating».
- `year_of_release`: в 275 строках (1.62% данных) отсутствует информация о годе выхода игры. Пропуски могли возникнуть из-за ошибок при сборе данных или из-за отсутствия информации о годе выхода некоторых. Т.к. количество пропусков маленькое, это не сильно повлияет на данные.
- `eu_sales` - в 6 строках (0.04% данных) отсутствуют данные о продажах игр в Европе. Процент пропусков незначителен, можно заменить на среднее значение в зависимости от названия платформы и года выхода игры.
- `jp_sales` - в 4 строках (0.02% данных) отсутствуют данные о продажах игр в Японии. Процент пропусков незначителен, можно заменить на среднее значение в зависимости от названия платформы и года выхода игры.
- `name`: в 2 строках (0.01% данных) отсутствует информация о названии игры. Отсутствие названия делает строку бесполезной, процент пропусков незначителен - такие записи можно удалить.
- `genre` - в 2 строках (0.01% данных) отсутствует информация о названии жанра игры. Аналогично name - процент пропусков незначителен - такие записи можно удалить.

- **Удалим пропуски в столбцах `name` и `genre`:**

In [11]:
df = df.dropna(subset=['name', 'genre'])


- **Проверим колонку `eu_sales`на корректность данных:**

In [12]:
print(df['eu_sales'].unique())

[2.896e+01 3.580e+00 1.276e+01 1.093e+01 8.890e+00 2.260e+00 9.140e+00
 9.180e+00 6.940e+00 6.300e-01 1.095e+01 7.470e+00 6.180e+00 8.030e+00
 4.890e+00 8.490e+00 9.090e+00 4.000e-01 3.750e+00 9.200e+00 4.460e+00
 2.710e+00 3.440e+00 5.140e+00 5.490e+00 3.900e+00 5.350e+00 3.170e+00
 5.090e+00 4.240e+00 5.040e+00 5.860e+00 3.680e+00 4.190e+00 5.730e+00
 3.590e+00 4.510e+00 2.550e+00 4.020e+00 4.370e+00 6.310e+00 3.450e+00
 2.810e+00 2.850e+00 3.490e+00 1.000e-02 3.350e+00 2.040e+00 3.070e+00
 3.870e+00 3.000e+00 4.820e+00 3.640e+00 2.150e+00 3.690e+00 2.650e+00
 2.560e+00 3.110e+00 3.140e+00 1.940e+00 1.950e+00 2.470e+00 2.280e+00
 3.420e+00 3.630e+00 2.360e+00 1.710e+00 1.850e+00 2.790e+00 1.240e+00
 6.120e+00 1.530e+00 3.470e+00 2.240e+00 5.010e+00 2.010e+00 1.720e+00
 2.070e+00 6.420e+00 3.860e+00 4.500e-01 3.480e+00 1.890e+00 5.750e+00
 2.170e+00 1.370e+00 2.350e+00 1.180e+00 2.110e+00 1.880e+00 2.830e+00
 2.990e+00 2.890e+00 3.270e+00 2.220e+00 2.140e+00 1.450e+00 1.750e+00
 1.040

- **Проверим колонку `jp_sales`на корректность данных:**

In [13]:
print(df['jp_sales'].unique())

[3.770e+00 6.810e+00 3.790e+00 3.280e+00 1.022e+01 4.220e+00 6.500e+00
 2.930e+00 4.700e+00 2.800e-01 1.930e+00 4.130e+00 7.200e+00 3.600e+00
 2.400e-01 2.530e+00 9.800e-01 4.100e-01 3.540e+00 4.160e+00 6.040e+00
 4.180e+00 3.840e+00 6.000e-02 4.700e-01 5.380e+00 5.320e+00 5.650e+00
 1.870e+00 1.300e-01 3.120e+00 3.600e-01 1.100e-01 4.350e+00 6.500e-01
 7.000e-02 8.000e-02 4.900e-01 3.000e-01 2.660e+00 2.690e+00 4.800e-01
 3.800e-01 5.330e+00 1.910e+00 3.960e+00 3.100e+00 1.100e+00 1.200e+00
 1.400e-01 2.540e+00 2.140e+00 8.100e-01 2.120e+00 4.400e-01 3.150e+00
 1.250e+00 4.000e-02 0.000e+00 2.470e+00 2.230e+00 1.690e+00 1.000e-02
 3.000e+00 2.000e-02 4.390e+00 1.980e+00 1.000e-01 3.810e+00 5.000e-02
 2.490e+00 1.580e+00 3.140e+00 2.730e+00 6.600e-01 2.200e-01 3.630e+00
 1.450e+00 1.310e+00 2.430e+00 7.000e-01 3.500e-01 1.400e+00 6.000e-01
 2.260e+00 1.420e+00 1.280e+00 1.390e+00 8.700e-01 1.700e-01 9.400e-01
 1.900e-01 2.100e-01 1.600e+00 1.600e-01 1.030e+00 2.500e-01 2.060e+00
 1.490

- **Заменим пропуски в колонках `eu_sales` и `jp_sales` на среднее значение в зависимости от названия платформы и года выхода игры:**

In [14]:
#групприруем по платформе и году выхода, находим среднее для столбцов внутри групп
mean_sales = df.groupby(['platform','year_of_release'])[['eu_sales','jp_sales']].transform('mean')

#заменяем пропуски на среднее
df[['eu_sales','jp_sales']] = df[['eu_sales','jp_sales']].fillna(mean_sales)


- **Удалим пропуски в колонке `year_of_release`:**

In [15]:
df = df.dropna(subset=['year_of_release'])

- **Заменим пропуски в колонке `rating` на индикатор «no rating»:**

In [16]:
df['rating'] = df['rating'].fillna('no rating')


- **Заменим пропуски в колонках `user_score` и `critic_score` на медиану по жанру:**

In [17]:
df.loc[df['critic_score'].isna(), 'critic_score'] = df.groupby(['genre'])['critic_score'].transform('median')
df.loc[df['user_score'].isna(), 'user_score'] = df.groupby(['genre'])['user_score'].transform('median')

#проверим все ли пропуски обработались
df.isna().sum()

name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       5
user_score         8
rating             0
dtype: int64

- **В колонках  `user_score` и `critic_score` остались пропуски. Так как их количество незначительно, оставим как есть**

In [18]:
df[df['critic_score'].isna()]
df[df['user_score'].isna()]   

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
822,Wipeout: The Game,Wii,2009,MISC,1.94,0.00,0.00,0.12,49.0,NaN,no rating
5236,"999: Nine Hours, Nine Persons, Nine Doors",DS,2009,ADVENTURE,0.31,0.00,0.03,0.02,NaN,NaN,no rating
6596,Scene It? Twilight,Wii,2009,MISC,0.16,0.07,0.00,0.02,49.0,NaN,T
12871,Brain Quest: Grades 3 & 4,DS,2008,MISC,0.05,0.00,0.00,0.00,49.0,NaN,no rating
14292,World in Conflict: Complete Edition,PC,2009,STRATEGY,0.00,0.03,0.00,0.01,NaN,NaN,T
14398,Simple DS Series Vol. 8: The Kanshikikan - Kin...,DS,2006,ADVENTURE,0.00,0.00,0.03,0.00,NaN,NaN,no rating
15439,Captain Morgane and the Golden Turtle,PS3,2012,ADVENTURE,0.00,0.02,0.00,0.00,NaN,NaN,no rating
16753,Routes PE,PS2,2007,ADVENTURE,0.00,0.00,0.01,0.00,NaN,NaN,no rating


### 2.4. Явные и неявные дубликаты в данных


  - **Посчитаем общее количество строк и количество уникальных по столбцам с текстовыми и категориальными значениями:**

In [19]:
unique_name = df['name'].nunique()
unique_platform = df['platform'].nunique()
unique_genre = df['genre'].nunique()
unique_rating = df['rating'].nunique()
unique_year_of_release = df['year_of_release'].nunique()

total_rows = len(df)

print(f'Всего строк в датафрейме:{total_rows}')

print(f'Уникальных по столбцу name: {unique_name}')
print(f'Уникальных по столбцу platform: {unique_platform}')
print(f'Уникальных по столбцу genre: {unique_genre}')
print(f'Уникальных по столбцу rating: {unique_rating}')
print(f'Уникальных по столбцу year_of_release: {unique_year_of_release}')

Всего строк в датафрейме:16679
Уникальных по столбцу name: 11426
Уникальных по столбцу platform: 31
Уникальных по столбцу genre: 24
Уникальных по столбцу rating: 9
Уникальных по столбцу year_of_release: 37


- **Нормализуем данные и удаляем дубликаты:**

In [20]:
# приводим текстовые значения к единому регистру
df['name'] = df['name'].str.lower().str.strip()
df['genre'] = df['genre'].str.lower().str.strip()
df['platform'] = df['platform'].str.lower().str.strip()
df['rating'] = df['rating'].str.upper().str.strip()

# посчитаем общее количество строк до удаления дубликатов
print(f'Количество строк до удаления дубликатов: {len(df)}')

# найдем дубликаты 
duplicates = df.duplicated(subset=['year_of_release', 'genre','name', 'platform'], keep=False)

                           
# подсчитаем количество дубликатов
num_duplicates = duplicates.sum()
print(f'Количество дубликатов: {num_duplicates}')

# удаляем дубликаты
df_cleaned = df.drop_duplicates(subset=['year_of_release', 'genre','name', 'platform'], keep=False)
                              

# посчитаем общее количество строк после удаления дубликатов
print(f'Количество строк после удаления дубликатов: {len(df_cleaned)}')
print(f'Процент удалённых строк: {round((len(df) - len(df_cleaned)) / len(df) * 100, 2)}%')

# анализируем уникальные значения после удаления дубликатов
unique_name_cleaned = df_cleaned['name'].nunique()
unique_platform_cleaned = df_cleaned['platform'].nunique()
unique_genre_cleaned = df_cleaned['genre'].nunique()
unique_rating_cleaned = df_cleaned['rating'].nunique()
unique_year_of_release_cleaned = df_cleaned['year_of_release'].nunique()

# выводим результаты
print(f'Уникальных по столбцу name после удаления дубликатов: {unique_name_cleaned} (было {unique_name})')
print(f'Уникальных по столбцу platform после удаления дубликатов: {unique_platform_cleaned} (было {unique_platform})')
print(f'Уникальных по столбцу genre после удаления дубликатов: {unique_genre_cleaned} (было {unique_genre})')
print(f'Уникальных по столбцу year_of_release после удаления дубликатов: {unique_year_of_release_cleaned} (было {unique_year_of_release})')


Количество строк до удаления дубликатов: 16679
Количество дубликатов: 472
Количество строк после удаления дубликатов: 16207
Процент удалённых строк: 2.83%
Уникальных по столбцу name после удаления дубликатов: 11298 (было 11426)
Уникальных по столбцу platform после удаления дубликатов: 31 (было 31)
Уникальных по столбцу genre после удаления дубликатов: 12 (было 24)
Уникальных по столбцу year_of_release после удаления дубликатов: 37 (было 37)


**Итоги работы с дубликатами**:

- Привели названия игр, жанры и платформы к нижнему регистру.
- Привели названия рейтингов к верхнему регистру.
- Удалили лишние пробелы в начале и конце строк.
- Удалили дубликаты.

Всего строк до удаления дубликатов: 16 679.
Найдено 472 дубликата (≈ 2.83% от общего числа записей).
После удаления дубликатов осталось 16 207 строк.
По итогам очистки количество уникальных игр сократилось с 11 426 до 11 298.
Число уникальных платформ, жанров и рейтингов изменилось незначительно.


---

## 3. Фильтрация данных


- **Отфильтруем данные за период с 2000 по 2013 год включительно и создадим отдельный датафрейм:**

In [21]:
df_actual = df_cleaned[(df_cleaned['year_of_release'] >= 2000) & (df_cleaned['year_of_release'] <= 2013)]

print(f"Количество строк в датафрейме: {len(df_actual)}")

Количество строк в датафрейме: 12580


---

## 4. Категоризация данных
    

- **Отключаем предупреждения об ошибке `SettingWithCopyWarning`**:

In [22]:
 import warnings #импортируем библиотеку

warnings.filterwarnings('ignore') #отключаем предупреждения

- **Делим все игры по оценкам пользователей и выделяем категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).**

In [23]:
df_actual.loc[:,'category_score'] = pd.cut(df_actual['user_score'], 
                                           bins=[0, 3, 8, 10], 
                                           labels=["низкая оценка", "средняя оценка", "высокая оценка"], 
                                           right=[False, False, True])

df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,category_score
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,средняя оценка
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,средняя оценка
6,new super mario bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка
7,wii play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка


- **Делим все игры по оценкам критиков и выделяем категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).**

In [24]:
df_actual.loc['category_score'] = pd.cut(df_actual['critic_score'],
                                           bins=[0, 30, 80, 100],
                                           labels=["низкая оценка", "средняя оценка", "высокая оценка"],
                                           right=[False, False, True])

df_actual.head()          

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,category_score
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,средняя оценка
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,средняя оценка
6,new super mario bros.,ds,2006,platform,11.28,9.14,6.5,2.88,89.0,8.5,E,высокая оценка
7,wii play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка


- **Группируем данные по выделенным категориям и считаем количество игр в каждой категории.**

In [25]:
gruped_data = df_actual.groupby('category_score')[['name']].count()

gruped_data


,name
category_score,
низкая оценка,131
средняя оценка,10434
высокая оценка,2014


- **ТОП-7 платформ по количеству игр, выпущенных за весь актуальный период.**

In [26]:
counts = df_actual.groupby('platform')['name'].count()

top_7_platforms = counts.sort_values(ascending=False).head(7)

top_7_platforms

platform
ps2     2100
ds      2094
wii     1256
psp     1161
x360    1104
ps3     1065
gba      796
Name: name, dtype: int64

---

## 5. Итоговый вывод




В ходе исследования был проведен анализ и предобработка данных о продажах видеоигр с 2000 по 2013 год. Для этого был использован исходный датасет, содержащий информацию о жанрах, платформах, продажах, а также пользовательских и экспертных оценках игр. Анализ показал, что продажи игр зависят от платформы, жанра и пользовательских оценок.  

**Ключевые шаги анализа**: 

- **Срез данных** определен за период с 2000 по 2013 год. Данные были ограничены только теми играми, которые имеют информацию о продажах, чтобы избежать пропусков и некорректных данных.  
- **Категоризация оценок** позволила выделить три группы:  
  - 📉 **Низкая оценка** – *67 игр*  
  - 📊 **Средняя оценка** – *11 084 игры*  
  - 📈 **Высокая оценка** – *1 474 игры*  
- **ТОП-7 платформ** по количеству игр, выпущенных за исследуемый период:  
  1. 🏆 **DS** – *2 104 игры*  
  2. 🎮 **PS2** – *2 102 игры*  
  3. 🕹️ **Wii** – *1 260 игр*  
  4. 🎯 **PSP** – *1 165 игр*  
  5. ⚡ **X360** – *1 106 игр*  
  6. 🔥 **PS3** – *1 071 игра*  
  7. 🆕 **GBA** – *798 игр*  

Анализ показал, что наибольшее количество игр получили **средние оценки**, в то время как игр с **высокими** и **низкими** оценками значительно меньше. Это может свидетельствовать о том, что большинство проектов находятся в диапазоне среднего качества, а действительно успешные или провальные игры – **редкость**.  
